In [5]:
import datetime
import numpy as np

def ctime2gps(time):
    gps_epoch=datetime.datetime(1980,1,6,0,0,0)
    given_time=datetime.datetime(time[0],time[1],time[2],time[3],time[4],int(time[5]))
    time_diff=given_time-gps_epoch
    total_seconds=time_diff.total_seconds()
    gps_week,gps_seconds=divmod(total_seconds,604800)
    return gps_week,gps_seconds


def find_near_time(obstime,lines):
    diff_time=[]
    time=[]
    for line in lines:
        if(line[0:2].strip().isdigit()):
            toc=[None]*6
            toc[0]=int(line[3:5].strip())
            toc[1]=int(line[6:8].strip())
            toc[2]=int(line[9:11].strip())
            toc[3]=int(line[12:14].strip())
            toc[4]=int(line[15:17].strip())
            toc[5]=float(line[18:22].strip())

            obs_week,obs_sec=ctime2gps(obstime)
            toc_week,toc_sec=ctime2gps(toc)

            diff=abs((obs_week-toc_week)*604800+obs_sec-toc_sec)
            diff_time.append(diff)
            time.append(toc)
        else:
            continue
    min_index=diff_time.index(min(diff_time))
    return time[min_index]

def caculate_pos_of_sat(matrix,a3,obstime,reftime):
    GM=3.986005*pow(10,5)
    n0=np.sqrt(GM)/pow(matrix[1,3],3)
    n=n0+matrix[0,2]

    # 计算归化时刻
    gps_obsweek,gps_obssec=ctime2gps(obstime)
    gps_refweek,gps_refseconds=ctime2gps(reftime)

    delta_t=a3[0]+a3[1]*(gps_obssec-gps_refseconds)+a3[2]*pow(gps_obssec-gps_refseconds,2)
    # t=gps_obssec-delta_t
    t=gps_refseconds-delta_t
    tk=t-matrix[2,0]

    Mk=matrix[0,3]+n*tk

    Mk=Mk%(np.pi*2)

    Ek=Mk
    Ek_old=Ek
    for i in range(10):
        Ek=Mk+matrix[1,1]*np.sin(Ek)
        if(abs(Ek-Ek_old)<1e-10):
            break
        else:
            Ek_old=Ek
    
    cos_Vk=(np.cos(Ek)-matrix[1,1])/(1-matrix[1,1]*np.cos(Ek))
    sin_Vk=(np.sqrt(1-matrix[1,1]*matrix[1,1])*np.sin(Ek))/(1-matrix[1,1]*np.cos(Ek))
    Vk=np.arctan(np.sqrt(1-matrix[1,1]*matrix[1,1])*np.sin(Ek)/(np.cos(Ek)-matrix[1,1]))

    phi_k=Vk+matrix[3,2]
    phi_k=phi_k%(np.pi*2)

    delta_u=matrix[1,0]*np.cos(2*phi_k)+matrix[1,2]*np.sin(2*phi_k)
    delta_r=matrix[3,1]*np.cos(2*phi_k)+matrix[0,1]*np.sin(2*phi_k)
    delta_i=matrix[2,1]*np.cos(2*phi_k)+matrix[2,3]*np.sin(2*phi_k)

    uk=phi_k+delta_u
    rk=pow(matrix[1,3],2)*(1-matrix[1,1]*np.cos(Ek))+delta_r
    ik=matrix[3,0]+delta_i+matrix[4,0]*tk

    xk=rk*np.cos(uk)
    yk=rk*np.sin(uk)

    we=7.29211567*pow(10,-5)
    omegek=matrix[2,2]+(matrix[3,3]-we)*tk-we*matrix[2,0]

    Xk=xk*np.cos(omegek)-yk*np.cos(ik)*np.sin(omegek)
    Yk=xk*np.sin(omegek)+yk*np.cos(ik)*np.cos(omegek)
    Zk=yk*np.sin(ik)

    return[Xk,Yk,Zk]


In [6]:

import numpy as np

obstime=[23,11,30,0,0,0]

# 文件路径
file_path='./data/al2h3340.23n'

with open(file_path,'r') as file:
    lines=file.readlines()

time=find_near_time(obstime,lines)

target_string='END OF HEADER'
HeadLine=0

for i,line in enumerate(lines,start=1):
    if target_string in line:
        HeadLine=i
        break

obs_type='GPS'
if('GPS' in lines[0]):
    obs_type='GPS'

sat_pos=[]
PRN_num=[]

for i in range (HeadLine,len(lines)):
    # 获取时间数据
    year=int(lines[i][3:5].strip())
    if(year==time[0]):
        month=int(lines[i][6:8].strip())
        day=int(lines[i][9:11].strip())
        hour=int(lines[i][12:14].strip())
        minute=int(lines[i][15:17].strip())
        sec=float(lines[i][18:22].strip())

        # 读取PRN号码
        PRN=int(lines[i][0:2].strip())
        if(month==time[1] and day==time[2] and hour==time[3] and minute==time[4] and sec==time[5]):
            # 读取卫星钟差改正参数
            time_change=[]
            a=float(lines[i][22:37].strip())*pow(10,int(lines[i][38:41].strip()))
            time_change.append(a)

            b=float(lines[i][41:56].strip())*pow(10,int(lines[i][57:60].strip()))
            time_change.append(b)

            c=float(lines[i][60:75].strip())*pow(10,int(lines[i][76:79].strip()))
            time_change.append(c)

            rows=6
            cols=4
            matrix=np.zeros((rows,cols))

            # 读取卫星位置计算的参数
            for j in range(0,rows):
                for k in range(0,cols):
                    matrix[j][k]=float(lines[i+1+j][3+19*k:18+19*k])*pow(10,int(lines[i+1+j][19+19*k:22+19*k]))
            # 计算卫星位置
            [sat_x,sat_y,sat_z]=caculate_pos_of_sat(matrix=matrix,a3=time_change,obstime=obstime,reftime=time)
            PRN_num.append(PRN)
            sat_pos.append([sat_x,sat_y,sat_z])

    else:
        continue

a=0
   